# FastAPI Notebook

Some getting-started stuff to the awesome world of FastAPI + Pydantic!

Some indications about this notebook:

- Must always run the first two cells

In [1]:
# Basic import of FastAPI and function to start running the API with Uvicorn

import uvicorn
from fastapi import FastAPI

app = FastAPI()

def run():
    uvicorn.run(app)

In [2]:
# Code in this cell is just for (re)starting the API on a Process, and other compatibility stuff with Jupyter cells.
# Just ignore it!

from multiprocessing import Process
from wait4it import wait_for

_api_process = None

def start_api():
    """Stop the API if running; Start the API; Wait until API (port) is available (reachable)"""
    global _api_process
    if _api_process:
        _api_process.terminate()
        _api_process.join()
    
    _api_process = Process(target=run, daemon=True)
    _api_process.start()
    wait_for(port=8000)

def delete_route(method: str, path: str):
    """Delete the given route from the API. This must be called on cells that re-define a route"""
    [app.routes.remove(route) for route in app.routes if method in route.methods and route.path == path]


## First endpoint

In [3]:
@app.get("/")
def get_root():
    return {"Hello": "World"}

start_api()

INFO:     Started server process [5304]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:51916 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:51916 - "GET /openapi.json HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [5304]


In [4]:
%%html
<iframe src="http://localhost:8000/docs#/default/get_root__get" width="800" height="600"></iframe>

## Endpoint with URL params

- Use Python string composition chars (`{abcd}`) as placeholders for URL params, in the endpoint route string
- Define them as function params (with the same name as used on the route)
- Typing the params will limit their input type

In [5]:
@app.get("/people/{person_id}")
def get_person(person_id: int):
    return {"person_id": person_id}

start_api()

INFO:     Started server process [5334]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:51942 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:51942 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:51950 - "GET /people/123abc HTTP/1.1" 422 Unprocessable Entity


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [5334]


In [6]:
%%html
<iframe src="http://localhost:8000/docs#/default/get_person_people__person_id__get" width="800" height="600"></iframe>

In [7]:
# Trying to set param as text with letters, when it MUST be int (unparseable)
import requests

r = requests.get("http://localhost:8000/people/123abc")
print("Status code:", r.status_code)
print("Response:", r.json())

Status code: 422
Response: {'detail': [{'loc': ['path', 'person_id'], 'msg': 'value is not a valid integer', 'type': 'type_error.integer'}]}


## Endpoint with query params

Function args that are not dict or Pydantic models, nor set on the URL, are treated as query params

In [8]:
from typing import Optional

@app.get("/people")
def list_people(surname: str, city: Optional[str] = None, limit: int = 20):
    return {
        "filters": {
            "surname": surname,
            "city": city
        },
        "limit": limit
    }

start_api()

INFO:     Started server process [5362]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:51974 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:51980 - "GET /people?surname=foolanito HTTP/1.1" 200 OK
INFO:     127.0.0.1:51974 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:51988 - "GET /people HTTP/1.1" 422 Unprocessable Entity
INFO:     127.0.0.1:51994 - "GET /people?surname=foolanito HTTP/1.1" 200 OK
INFO:     127.0.0.1:52000 - "GET /people HTTP/1.1" 422 Unprocessable Entity


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [5362]


In [9]:
%%html
<iframe src="http://localhost:8000/docs#/default/list_people_people_get" width="800" height="600"></iframe>

In [12]:
# Get with "surname" param only
import requests

r = requests.get("http://localhost:8000/people?surname=foolanito")
print("Status code:", r.status_code)
print("Response:", r.json())

Status code: 200
Response: {'filters': {'surname': 'foolanito', 'city': None}, 'limit': 20}


In [13]:
# Get without params ("surname" is required)
import requests

r = requests.get("http://localhost:8000/people")
print("Status code:", r.status_code)
print("Response:", r.json())

Status code: 422
Response: {'detail': [{'loc': ['query', 'surname'], 'msg': 'field required', 'type': 'value_error.missing'}]}


## Endpoint with body + model

- Using Pydantic BaseModel
- Set as function param (typing it as the BaseModel class)

In [14]:
from pydantic import BaseModel
from typing import Optional

class Person(BaseModel):
    name: str
    age: Optional[int]
    profession: str = "Student"

@app.post("/people")
def create_person(person: Person):
    return person.dict()

start_api()

INFO:     Started server process [5399]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:52024 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:52024 - "GET /openapi.json HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [5399]


Try to:

- Send age = "5"
- Send age = "5x"
- Send no age
- Send no name
- Send no profession
- Send custom profession

In [15]:
%%html
<iframe src="http://localhost:8000/docs#/default/create_person_people_post" width="800" height="600"></iframe>

## Improving models & their documentation

- Add a description as docstring
- Set the value of fields to Field objects - to define the field description, constraints and more!

Try to:

- Send an age that is less than 18 years old
- Send an empty string as age or profession

In [16]:
from pydantic import BaseModel, Field
from typing import Optional

delete_route("POST", "/people")

class Person(BaseModel):
    """Just a human being, with a name, age and profession"""
    name: str = Field(..., description="The name of this person", example="Anna", min_length=1)
    age: Optional[int] = Field(None, description="The age of this person in years. Must be 18 years or older", example=18, ge=18)  # (g)reater or (e)qual 18
    profession: str = Field("Student", description="The profession of this person", example="Engineer", min_length=1)

@app.post("/people")
def create_person(person: Person):
    return person.dict()

start_api()

INFO:     Started server process [5425]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:52050 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:52050 - "GET /openapi.json HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [5425]


In [17]:
%%html
<iframe src="http://localhost:8000/docs#/default/create_person_people_post" width="800" height="600"></iframe>

## Improving query params

Query params can also be fully documented and have additional validators.

In [18]:
from typing import Optional
from fastapi import Query

delete_route("GET", "/people")

@app.get("/people")
def list_people(
        surname: str = Query(..., description="Surname to filter by"),
        city: Optional[str] = Query(None, description="City to filter by"),
        limit: float = Query(20, description="Max results returned", ge=1, le=1000)
):
    return {
        "filters": {
            "surname": surname,
            "city": city
        },
        "limit": limit
    }

start_api()

INFO:     Started server process [5451]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:52076 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:52082 - "GET /people?surname=Fulanito&limit=0 HTTP/1.1" 422 Unprocessable Entity
INFO:     127.0.0.1:52088 - "GET /people?surname=Fulanito&limit=1001 HTTP/1.1" 422 Unprocessable Entity


In [19]:
%%html
<iframe src="http://localhost:8000/docs#/default/list_people_people_get" width="800" height="600"></iframe>

In [20]:
# Get with limit=0 and limit=1001
import requests

for limit in [0, 1001]:
    r = requests.get(f"http://localhost:8000/people?surname=Fulanito&limit={limit}")
    print(f"limit={limit}")
    print("\tStatus code:", r.status_code)
    print("\tResponse:", r.json())

limit=0
	Status code: 422
	Response: {'detail': [{'loc': ['query', 'limit'], 'msg': 'ensure this value is greater than or equal to 1', 'type': 'value_error.number.not_ge', 'ctx': {'limit_value': 1}}]}
limit=1001
	Status code: 422
	Response: {'detail': [{'loc': ['query', 'limit'], 'msg': 'ensure this value is less than or equal to 1000', 'type': 'value_error.number.not_le', 'ctx': {'limit_value': 1000}}]}
